In [50]:
import json
import pandas as pd
import sys

def to_df(filename):
    with open(filename, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)

    df = pd.DataFrame(data)
    df.to_csv(f'{filename.split(".json")[0]}.csv', index=False, sep=';')
    return df
scan = to_df('coin_data.json')
with open("contract_data.json", 'r') as json_file:
        data = json.load(json_file)


In [58]:

# Функция для извлечения информации о монете из словарей
def extract_coin_info(item):
    if item is not None and 'props' in item and 'pageProps' in item['props']:
        return item['props']['pageProps']
    else:
        return None

# Создание нового датафрейма на основе информации о монете
coin_df = pd.json_normalize(scan['items'].apply(extract_coin_info).dropna())
coin_df

,rank,key,name,hasFundingRounds,symbol,type,lifeCycle,unlimitedSupply,totalSupply,percentOfCircSupply,...,histPrices.3Y.BTC,histPrices.3Y.ETH,histPrices.3Y.percentUSD,icoData.kyc,icoData.accepting,icoData.whitelist,icoData.teamFromCountries,athPrice.dateUSD,icoData.rates,isSponsored
0,NaN,patex,Patex,False,PATEX,token,crowdsale,False,8.000000e+06,4.21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1716.0,cyber-arena,Cyber Arena,False,CAT,token,traded,False,5.000000e+09,14.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1699.0,virtual-versions,Virtual Versions,False,VV,token,traded,False,1.000000e+09,11.29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1595.0,kryptonite,Kryptonite,False,SEILOR,token,traded,False,1.000000e+09,19.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1120.0,spartadex,SpartaDEX,False,SPARTA,token,traded,False,1.000000e+08,18.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2715,NaN,nft-alley,NFT Alley,True,ALLEY,token,traded,False,1.000000e+07,86.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2716,NaN,algopainter,AlgoPainter,False,ALGOP,token,inactive,False,1.000000e+08,10.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2717,1365.0,mydefipet,MyDeFiPet,True,DPET,token,traded,False,1.000000e+08,50.23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2718,1814.0,mist-nft,Mist,False,MIST,token,traded,False,1.000000e+09,5.64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
unique_keys_count = coin_df['key'].nunique()
print("Number of unique values in the 'key' column:", unique_keys_count)

Number of unique values in the 'key' column: 1424


In [73]:
threshold = 0.1 * len(coin_df)
df_cleaned = coin_df.dropna(axis=1, thresh=threshold)
yes_columns = ['key', 'name', 'hasFundingRounds', 'links', 'volume24hUsd', 'totalSupply', 'percentOfCircSupply', ]
df_cleaned = df_cleaned[yes_columns]
df_cleaned

,key,name,hasFundingRounds,links,volume24hUsd,totalSupply,percentOfCircSupply
0,patex,Patex,False,"[{'type': 'web', 'value': 'https://patex.io/'}...",NaN,8.000000e+06,4.21
1,cyber-arena,Cyber Arena,False,"[{'type': 'web', 'value': 'https://www.cyber-a...",29920.120165,5.000000e+09,14.95
2,virtual-versions,Virtual Versions,False,"[{'type': 'web', 'value': 'https://virtualvers...",663475.492992,1.000000e+09,11.29
3,kryptonite,Kryptonite,False,"[{'type': 'discord', 'value': 'https://discord...",36897.969515,1.000000e+09,19.30
4,spartadex,SpartaDEX,False,"[{'type': 'web', 'value': 'https://spartadex.i...",34726.685377,1.000000e+08,18.70
...,...,...,...,...,...,...,...
2715,nft-alley,NFT Alley,True,"[{'type': 'web', 'value': 'https://nftalley.io...",11.203164,1.000000e+07,86.99
2716,algopainter,AlgoPainter,False,"[{'type': 'web', 'value': 'https://www.algopai...",NaN,1.000000e+08,10.00
2717,mydefipet,MyDeFiPet,True,"[{'type': 'whitepaper', 'value': 'https://www....",68614.753369,1.000000e+08,50.23
2718,mist-nft,Mist,False,"[{'type': 'web', 'value': 'https://mist.game/'...",71808.588700,1.000000e+09,5.64


In [77]:
n_data = list()
n_data = []
for key in data:
    if data[key] == []:
        continue
    n_data.append({'name' : key, 'code' : data[key]['result'][0]})


In [81]:
codes = pd.DataFrame(n_data)

In [83]:
codes

,name,code
0,Patex,{'SourceCode': '// OpenZeppelin Contracts v4.4...
1,Cyber Arena,{'SourceCode': '// File: @openzeppelin/contrac...
2,Virtual Versions,"{'SourceCode': '{{ ""language"": ""Solidity"", ..."
3,Kryptonite,"{'SourceCode': '{{ ""language"": ""Solidity"", ..."
4,SpartaDEX,"{'SourceCode': '{{ ""language"": ""Solidity"", ..."
...,...,...
1166,Polylastic,"{'SourceCode': '{{ ""language"": ""Solidity"", ..."
1167,NFT Alley,{'SourceCode': '// File: openzeppelin-solidity...
1168,AlgoPainter,{'SourceCode': '// File: @openzeppelin/contrac...
1169,Mist,"{'SourceCode': '{{ ""language"": ""Solidity"", ..."


In [85]:
result_df = pd.merge(codes, df_cleaned.drop_duplicates('name'), on='name', how='left')
result_df

,name,code,key,hasFundingRounds,links,volume24hUsd,totalSupply,percentOfCircSupply
0,Patex,{'SourceCode': '// OpenZeppelin Contracts v4.4...,patex,False,"[{'type': 'web', 'value': 'https://patex.io/'}...",NaN,8.000000e+06,4.21
1,Cyber Arena,{'SourceCode': '// File: @openzeppelin/contrac...,cyber-arena,False,"[{'type': 'web', 'value': 'https://www.cyber-a...",29920.120165,5.000000e+09,14.95
2,Virtual Versions,"{'SourceCode': '{{ ""language"": ""Solidity"", ...",virtual-versions,False,"[{'type': 'web', 'value': 'https://virtualvers...",663475.492992,1.000000e+09,11.29
3,Kryptonite,"{'SourceCode': '{{ ""language"": ""Solidity"", ...",kryptonite,False,"[{'type': 'discord', 'value': 'https://discord...",36897.969515,1.000000e+09,19.30
4,SpartaDEX,"{'SourceCode': '{{ ""language"": ""Solidity"", ...",spartadex,False,"[{'type': 'web', 'value': 'https://spartadex.i...",34726.685377,1.000000e+08,18.70
...,...,...,...,...,...,...,...,...
1166,Polylastic,"{'SourceCode': '{{ ""language"": ""Solidity"", ...",polylastic,True,"[{'type': 'web', 'value': 'https://polylastic....",293938.399090,9.825410e+10,72.91
1167,NFT Alley,{'SourceCode': '// File: openzeppelin-solidity...,nft-alley,True,"[{'type': 'web', 'value': 'https://nftalley.io...",11.203164,1.000000e+07,86.99
1168,AlgoPainter,{'SourceCode': '// File: @openzeppelin/contrac...,algopainter,False,"[{'type': 'web', 'value': 'https://www.algopai...",NaN,1.000000e+08,10.00
1169,Mist,"{'SourceCode': '{{ ""language"": ""Solidity"", ...",mist-nft,False,"[{'type': 'web', 'value': 'https://mist.game/'...",71808.588700,1.000000e+09,5.64
